In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0
# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6M0x3NG5dGXOJK4jW2jsprymfUHFFTyizls8TOseoTYw',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'nutritionimageprocessingusingcnn-donotdelete-pr-oaym8kuyz8vgj7'
object_key = 'TRAIN_SET.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [ ]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [ ]:
ls

In [ ]:
pwd

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of Layers
from tensorflow.keras import layers #A Layer consists of a tensor-in tensor-out computation function
#Dense Layer is the regular deeply connected neural network Layer
from tensorflow.keras.layers import Dense, Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
x_train = train_datagen.flow_from_directory(
    r'/home/wsuser/work/TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
x_test = test_datagen.flow_from_directory(
    r'/home/wsuser/work/TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

Found 2626 images belonging to 5 classes.
Found 2626 images belonging to 5 classes.

In [ ]:
print(x_train.class_indices)#checking the number of classes

In [ ]:
print(x_test.class_indices) #checking the number of classes

In [ ]:
from collections import Counter as c 
c(x_train .labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

In [ ]:
model=Sequential()

In [ ]:
# Initializing the CNN classifier = Sequential()
classifier = Sequential()
# First convolution layer and pooling
classifier.add(Conv2D(32,(3, 3), input_shape=(64, 64, 3),activation='relu')) 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Flattening the Layers
classifier.add(Flatten())
# Adding fully connected Layer a
classifier.add(Dense (units=128, activation='relu'))
classifier.add(Dense (units=5, activation='softmax')) # softmax for more than 2

In [ ]:
classifier.summary()

In [ ]:
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
##Fitting the model
classifier.fit_generator(
    generator=x_train,steps_per_epoch = len(x_train), 
    epochs=20,validation_data=x_test,validation_steps = len(x_test)) # No of images in test set

In [ ]:
1s

nutrition.h5  TRAIN_SET/

In [ ]:
classifier.save('nutrition.h5')

nutrition.h5  TRAIN_SET/

In [ ]:
pwd

'/home/wsuser/work'

In [ ]:
### Predicting our results
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("nutrition.h5") #Loading the model for testing
from tensorflow.keras.preprocessing import image

In [ ]:
img =image.load_img(r"/home/wsuser/work/TRAIN_SET/ORANGE/100_100.jpg",grayscale=False, target_size= (64,64))
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis=0) #changing the shape =
y =np.argmax(model.predict(x),axis=1)
index=['APPLES', 'BANANA', 'ORANGE','PINEAPPLE','WATERMELON']
index[y[0]]

'ORANGE'
**IBM Deployment**

In [ ]:
!pip install watson-machine-learning-client

In [ ]:
!pip install keras==2.2.4
!pip install tensorflow==2.5.0

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"BGB_wP_pV2ZAoyBd0J0tJ8SYUf1XpCvuzPwkefw4XhCn"
}

client=APIClient(wml_credentials)

In [ ]:
client

In [ ]:
def guid_space_name(client,nutrition_deploy):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==nutrition_deploy)['metadata']['id'])


DEPLOYMENT





In [ ]:
space_uid=guid_space_name(client,'nutrition_deploy')
print("Space UID " + space_uid)

In [ ]:
client.set.default_space(space_uid)

'SUCCESS'

In [ ]:
client.software_specifications.list(200)

In [ ]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')

In [ ]:
software_space_uid

In [ ]:
!tar -zcvf nutrition-classification-model.tgz nutrition.h5

nutrition.h5

In [ ]:
model_details=client.repository.store_model(model ='nutrition-classification-model.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN Model Building",
    client.repository.ModelMetaNames.TYPE: 'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_space_uid
})

In [ ]:
model_details=client.repository.get_model_id(model_details)

In [ ]:
model_id

In [ ]:
1S

In [ ]:
client.repository.download(model_id,'nutrition.tar.gb')

In [ ]:
pwd